# Tweet Scraper

Este es el notebook que usaremos para el scrapeo de datos


## Librerias

Para el scrapeo usaremos las siguientes librerias:
* Sntwitter
* Pandas

In [ ]:
import datetime
from datetime import timedelta

import snscrape.modules.twitter as snstwitter

import pandas as pd
import seaborn as sns

from tqdm import tqdm

from src.JATS import JATS

## Parametros



In [ ]:
from JATS.src.JATS.analyzer import Analyzer
from nltk.corpus import stopwords
a = Analyzer()
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

In [ ]:
text = "Hi, today I hate bitcoin and all their products. Call me Mark Tyson"
print(" ".join([w for w in text.split() if not w in stop_words]))

In [ ]:
%%timeit text = "Hi, today I hate bitcoin and all their products"
filtered_sentence = " ".join([w for w in text.split() if not w in stop_words])

a.get_sentiment(filtered_sentence)

In [ ]:
%%timeit text = "Hi, today I hate bitcoin and all their products"
a.get_sentiment(text)

In [ ]:
query = '"BTC" OR "bitcoin"'

date_from = datetime.date(2018, 4, 1)
date_until = datetime.date(2018, 4, 30)

tweet_list = JATS.get_tweets(query, date_from, date_until, verbose = True)

In [ ]:
columnNames = [
    'Datetime',
    'Tweet Id',
    'Text', 
    'NumReplies',
    'NumRetweets',
    'NumLikes', 
    'IDOriginalRetweeted', 
    'Username',
    'isVerified'
]
tweet_df = pd.DataFrame(tweet_list, columns=columnNames)

### Lectura de ficheros ya existentes

In [ ]:
file_name = "data/tweets/2018-03-02/2018-04-03/tweet_list.csv"
tweet_df = pd.read_csv(file_name, sep=';')

tweet_df["Datetime"] = pd.to_datetime(tweet_df["Datetime"])

## Analisis de sentimientos

### Preparación del Analisis

En caso de ser nuestra primera ejecución, deberemos instalar un conjunto de datasets utiles para *nltk*.

In [ ]:
from JATS.src.JATS.analyzer import Analyzer

## Analisis de Sentimiento

Eliminaremos los valores nulos ya que parece que cuando el algoritmo no es capaz de determinar el sentimiento, tiende a ponerle un 0, creando una desviación del sentimiento real.

Lo primero que haremos será mostrar la **media del sentimiento** y una **gráfica de distribución del sentimiento**.

In [ ]:
a = Analyzer()

In [ ]:
a.analyze(tweet_df, "data/tweets/2018-04-01/2018-04-02") # Saved to a CSV

## Analisis de Similitudes

Tenemos que comprobar la existencia de tweets similares para evitar el SPAM que existe en mensajes que no son completamente identicos.

Para ellos haremos uso de la metrica de similitud Cosine Similarity y despues aplicaremos un DBScan para asignar clusters a esos tweets.

In [ ]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

def get_cosine_similarity(cleaned_texts):
    vectorizer = CountVectorizer().fit_transform(cleaned_texts)
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)

In [ ]:

print(tweet_df.isnull().sum())
tweet_df = tweet_df.dropna(axis=0, subset=['Text'])
print(tweet_df.isnull().sum())

csim = get_cosine_similarity(tweet_df['Text'])

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
clustering = DBSCAN(eps=1.04, min_samples=1).fit(csim)
unique_elements, counts_elements = np.unique(clustering.labels_, return_counts=True)
print(type(clustering.labels_))
print(np.asarray((unique_elements, counts_elements)))

In [ ]:
print(csim)

In [ ]:
tweet_df['Prediction'] = clustering.labels_.tolist()
df = pd.DataFrame
tweet_df[tweet_df['Prediction'] < 1].head()

In [ ]:
import pandas as pd

column = ["DateTime","Cosa"]
df = pd.DataFrame(columns=column)

In [ ]:
df.count().reset_index()

In [ ]:
class Foo: pass
class Bar(Foo): pass
class Bar2(Foo): pass
class Bar(Bar): pass

In [ ]:
Foo.__subclasses__()

In [ ]:
[ q.__name__ for q in Foo.__subclasses__()]


In [ ]:
[ q() for q in Foo.__subclasses__() if q.__name__ == "Bar"]

In [ ]:
import pandas as pd


d = {'col1': [1, 2,1,2,4,7,1], 
     'col2': [1, 2,1,1,4,1,1],
     'col3': [11, 32,41,14,4,4,18],
     'col5': ['11', '32','41','14','4','4','18']}
df = pd.DataFrame(data=d)

In [ ]:
df.groupby('col1', as_index=False).mean().columns



## Separar obtención de bitcoin a dia a dia

In [75]:
import os
from pathlib import Path
import pandas as pd


In [81]:
df = pd.read_csv("JABA/data/bitcoin/BTCUSDT-1m-data.csv", sep=",")

In [82]:
df["timestamp"] = pd.to_datetime(df["timestamp"])

df["timestamp_round"] = df["timestamp"].dt.floor("d")
df = df.set_index("timestamp_round")

In [83]:

DFList = []
i = 0
datee = 0
for group in df.groupby(df.index):
    
    i += 1
    datee = str(group[1].iloc[0]["timestamp"].to_pydatetime().date())
    path = bitcoin_dataset_file_name = os.path.join(base_dir_btc, datee)
    file_name = os.path.join(path, "bitcoin.csv")
    Path(path).mkdir(parents=True, exist_ok=True)
    if i%100 == 0: 
        print(file_name)
        print(f"Paso {i%365} del año {i//365}")
    group[1].to_csv(file_name, sep=";")

JABA/data/bitcoin/2017-11-24\bitcoin.csv
Paso 100 del año 0
JABA/data/bitcoin/2018-03-04\bitcoin.csv
Paso 200 del año 0
JABA/data/bitcoin/2018-06-12\bitcoin.csv
Paso 300 del año 0
JABA/data/bitcoin/2018-09-20\bitcoin.csv
Paso 35 del año 1
JABA/data/bitcoin/2018-12-29\bitcoin.csv
Paso 135 del año 1
JABA/data/bitcoin/2019-04-08\bitcoin.csv
Paso 235 del año 1
JABA/data/bitcoin/2019-07-17\bitcoin.csv
Paso 335 del año 1
JABA/data/bitcoin/2019-10-25\bitcoin.csv
Paso 70 del año 2
JABA/data/bitcoin/2020-02-02\bitcoin.csv
Paso 170 del año 2
JABA/data/bitcoin/2020-05-12\bitcoin.csv
Paso 270 del año 2
JABA/data/bitcoin/2020-08-20\bitcoin.csv
Paso 5 del año 3
JABA/data/bitcoin/2020-11-28\bitcoin.csv
Paso 105 del año 3
JABA/data/bitcoin/2021-03-08\bitcoin.csv
Paso 205 del año 3
JABA/data/bitcoin/2021-06-16\bitcoin.csv
Paso 305 del año 3


AttributeError: 'DataFrameGroupBy' object has no attribute 'shape'